In [112]:
import pandas as pd
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from gensim import matutils,corpora, models
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import warnings


In [113]:
negative_list = os.listdir("negative_reviews") 
positive_list = os.listdir("positive_reviews")

In [114]:
print(negative_list)

['deceptive_from_MTurk', 'truthful_from_Web']


In [115]:

def preprocess(files_list,root_dir,review):
    listOfFiles = []
    for (dirpath, dirnames, filenames) in os.walk(root_dir):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
    labeled_class = []
    reviews = []
    actual_class =[]
    for j in listOfFiles:
        labeled_class.append(review)
        k = str(open(j,encoding='utf-8').read())
        reviews.append(k)
        actual_class.append(str(j.split('\\')[1].split('_')[0]))
    data = pd.DataFrame({'labeled_class':labeled_class,'review':reviews,'actual_class':actual_class})
    return data
        

In [116]:
print(positive_list)

['deceptive_from_MTurk', 'truthful_from_TripAdvisor']


In [117]:
def preprocess(files_list,root_dir,review):
    labeled_class = []
    reviews = []
    actual_class =[]
    for j in files_list:
        labeled_class.append(review)
        k = str(open(root_dir + '/' + j,encoding='utf-8').read())
        reviews.append(k)
        actual_class.append(str(j.split('_')[0]))
    data = pd.DataFrame({'labeled_class':labeled_class,'review':reviews,'actual_class':actual_class})
    return data

In [118]:
negative_df = preprocess(negative_list,'negative_reviews','negative')
positive_df = preprocess(positive_list,'positive_reviews','positive')

PermissionError: [Errno 13] Permission denied: 'negative_reviews/deceptive_from_MTurk'

In [71]:
negative_df.head()

,labeled_class,review,actual_class
0,negative,We stayed at the Schicago Hilton for 4 days an...,deceptive
1,negative,Hotel is located 1/2 mile from the train stati...,deceptive
2,negative,I made my reservation at the Hilton Chicago be...,deceptive
3,negative,"When most people think Hilton, they think luxu...",deceptive
4,negative,My husband and I recently stayed stayed at the...,deceptive


In [72]:
negative_df['actual_class'].value_counts()

actual_class
deceptive    400
truthful     400
Name: count, dtype: int64

In [73]:
negative_df['labeled_class'].value_counts()

labeled_class
negative    800
Name: count, dtype: int64

In [74]:
target = []
for i in positive_df.index:
    if ((positive_df['labeled_class'][i] == 'positive') & (positive_df['actual_class'][i] == 'truthful')):
        target.append(2)
    elif ((positive_df['labeled_class'][i] == 'positive') & (positive_df['actual_class'][i] == 'deceptive')):
        target.append(1)
    else:
        print('Error!')
positive_df['target'] = target

In [75]:
target = []
for i in negative_df.index:
    if ((negative_df['labeled_class'][i] == 'negative') & (negative_df['actual_class'][i] == 'truthful')):
        target.append(3)
    elif ((negative_df['labeled_class'][i] == 'negative') & (negative_df['actual_class'][i] == 'deceptive')):
        target.append(4)
    else:
        print('Error!')
negative_df['target'] = target

In [76]:
data = positive_df.merge(negative_df,how='outer')
data = data[['review','target']]

In [77]:
data.head()

,review,target
0,""" Barely Average Hotel at Premium Price"" This ...",3
1,A decent place to stay. The people are nice en...,4
2,A disappointment to say the least. I have hear...,3
3,A few weeks ago we stayed at the Hard Rock Hot...,4
4,A friend and I stayed at the Hyatt Regency in ...,4


In [78]:
data.target.value_counts()

target
3    400
4    400
1    400
2    400
Name: count, dtype: int64

In [79]:
def extract_tokens(df):
    review_tokenized = []
    lmt = WordNetLemmatizer()
    for index, datapoint in df.iterrows():
        tokenize_words = word_tokenize(datapoint["review"].lower(),language='english')
        pos_word = pos_tag(tokenize_words)
        tokenize_words = ["_".join([lmt.lemmatize(i[0]),i[1]]) for i in pos_word if (i[0] not in stopwords.words("english") and len(i[0]) > 2)]
        token = ' '.join(tokenize_words)
        review_tokenized.append(token)
    df["review_tokenized"] = review_tokenized
    return df
data = extract_tokens(data)

'def extract_tokens(df):\n    review_tokenized = []\n    lmt = WordNetLemmatizer()\n    for index, datapoint in df.iterrows():\n        tokenize_words = word_tokenize(datapoint["review"].lower(),language=\'english\')\n        pos_word = pos_tag(tokenize_words)\n        tokenize_words = ["_".join([lmt.lemmatize(i[0]),i[1]]) for i in pos_word if (i[0] not in stopwords.words("english") and len(i[0]) > 2)]\n        token = \' \'.join(tokenize_words)\n        review_tokenized.append(token)\n    df["review_tokenized"] = review_tokenized\n    return df\ndata = extract_tokens(data)'

In [80]:
data.head()

,review,target
0,""" Barely Average Hotel at Premium Price"" This ...",3
1,A decent place to stay. The people are nice en...,4
2,A disappointment to say the least. I have hear...,3
3,A few weeks ago we stayed at the Hard Rock Hot...,4
4,A friend and I stayed at the Hyatt Regency in ...,4


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [82]:
vector = TfidfVectorizer( lowercase=False,stop_words='english')
cntvct = vector.fit_transform(data['review'])

In [83]:
print(cntvct)

  (0, 498)	0.13492819188201385
  (0, 456)	0.1483040357928136
  (0, 1288)	0.0893308352918609
  (0, 1907)	0.14047966868949635
  (0, 1913)	0.1306221319342994
  (0, 2351)	0.04880391708779805
  (0, 6157)	0.1438838366733525
  (0, 3282)	0.06326641252425022
  (0, 6826)	0.16703647389908524
  (0, 5317)	0.2809593373789927
  (0, 6950)	0.13492819188201385
  (0, 9474)	0.10817650406041443
  (0, 9508)	0.04055783527814303
  (0, 759)	0.16732010665056668
  (0, 911)	0.1483040357928136
  (0, 1367)	0.14047966868949635
  (0, 1277)	0.1483040357928136
  (0, 6881)	0.0738025165700864
  (0, 6859)	0.06410817117795792
  (0, 5570)	0.08470340275046266
  (0, 9509)	0.11248575334159792
  (0, 5285)	0.06978295360420038
  (0, 4708)	0.06866163533081912
  (0, 9737)	0.0919990446243702
  (0, 2863)	0.10117342105856283
  :	:
  (1599, 10304)	0.13534022596749504
  (1599, 9948)	0.08361448874332202
  (1599, 5149)	0.09311067288154837
  (1599, 5066)	0.09891612418919689
  (1599, 6233)	0.1535310795824604
  (1599, 6076)	0.150343935684548

In [84]:
cntvct.shape

(1600, 10839)

In [85]:
vector.get_feature_names()

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [106]:
vector = TfidfVectorizer( lowercase=False,stop_words='english',max_features=2000)
cntvct_token = vector.fit_transform(data["review_tokenized"])

In [107]:
data.dtypes

review              object
target               int64
review_tokenized    object
dtype: object

In [108]:
cntvct_token

<1600x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 81018 stored elements in Compressed Sparse Row format>

In [109]:
print(cntvct_token)

  (0, 93)	0.1364047373656643
  (0, 1426)	0.11039160645661797
  (0, 1987)	0.0704308732637901
  (0, 1337)	0.1389460344251074
  (0, 904)	0.20144813194698022
  (0, 186)	0.12644229298674134
  (0, 24)	0.1550210547827095
  (0, 76)	0.16036620161475662
  (0, 1539)	0.07803712989209645
  (0, 861)	0.04474814849341855
  (0, 990)	0.09288320072309046
  (0, 1547)	0.40289626389396044
  (0, 1602)	0.16937260625950443
  (0, 1032)	0.28995721419576137
  (0, 1632)	0.14096877050895668
  (0, 1397)	0.14421772852535136
  (0, 181)	0.1047373517617891
  (0, 1675)	0.22268074696654871
  (0, 736)	0.1597969599098303
  (0, 1801)	0.19824435102256815
  (0, 1670)	0.08075540064881251
  (0, 442)	0.08884482593784591
  (0, 1953)	0.17831067883519375
  (0, 729)	0.12459021821527022
  (0, 107)	0.2297889789747251
  :	:
  (1599, 1472)	0.1770023767900114
  (1599, 1561)	0.18927931209497975
  (1599, 1301)	0.13543547621991633
  (1599, 451)	0.1726578303722536
  (1599, 1816)	0.17476423675322308
  (1599, 1600)	0.11443374848114343
  (1599, 

In [110]:
cntvct_token.shape

(1600, 2000)

In [111]:
vector.get_feature_names()

['00_CD',
 '10',
 '100_CD',
 '11',
 '15',
 '200_CD',
 '24',
 '2_CD',
 '300_CD',
 '30_CD',
 '30am_CD',
 '30pm_CD',
 '3pm_CD',
 '3rd_CD',
 '400_CD',
 '50_CD',
 '5_CD',
 '5th_JJ',
 '6th_CD',
 '_CD',
 '_IN',
 '_JJ',
 '_NN',
 'able_JJ',
 'absolutely_RB',
 'accepted_VBD',
 'access_NN',
 'accessible_JJ',
 'accommodate_VB',
 'accommodating_VBG',
 'accommodation_NNS',
 'accomodations_NNS',
 'account_NN',
 'across_IN',
 'acted_VBD',
 'action_NN',
 'activity_NNS',
 'actual_JJ',
 'actually_RB',
 'add_VB',
 'added_JJ',
 'addition_NN',
 'additional_JJ',
 'additionally_RB',
 'adequate_JJ',
 'adjacent_JJ',
 'adult_NNS',
 'advance_NN',
 'advantage_NN',
 'advertised_VBN',
 'advisor_NN',
 'affinia_JJ',
 'affinia_NN',
 'affordable_JJ',
 'afternoon_NN',
 'ago_RB',
 'agreed_VBD',
 'ahead_RB',
 'air_NN',
 'airport_NN',
 'alarm_NN',
 'allegro_NN',
 'alley_NN',
 'allow_VBP',
 'allowed_VBN',
 'almost_RB',
 'alone_RB',
 'along_IN',
 'along_RB',
 'already_RB',
 'also_RB',
 'although_IN',
 'always_RB',
 'amalfi_JJ

In [112]:
feature_names = vector.get_feature_names()
for col in cntvct_token.nonzero()[1]:
    print(feature_names[col], ' - ', cntvct_token[0, col])

anyone_NN  -  0.1364047373656643
recommend_VB  -  0.11039160645661797
would_MD  -  0.0704308732637901
pool_NN  -  0.1389460344251074
indoor_JJ  -  0.20144813194698022
beautiful_JJ  -  0.12644229298674134
absolutely_RB  -  0.1550210547827095
amazing_JJ  -  0.16036620161475662
service_NN  -  0.07803712989209645
hotel_NN  -  0.04474814849341855
like_IN  -  0.09288320072309046
shampoo_NN  -  0.40289626389396044
smell_NN  -  0.16937260625950443
loved_VBD  -  0.28995721419576137
spacious_JJ  -  0.14096877050895668
quite_RB  -  0.14421772852535136
bathroom_NN  -  0.1047373517617891
staying_VBG  -  0.22268074696654871
friend_NNS  -  0.1597969599098303
touch_NN  -  0.19824435102256815
stay_VB  -  0.08075540064881251
could_MD  -  0.08884482593784591
wifi_NN  -  0.17831067883519375
free_JJ  -  0.12459021821527022
appreciated_VBD  -  0.2297889789747251
also_RB  -  0.09875666716213397
soft_JJ  -  0.19528726829590773
immaculate_JJ  -  0.22052278218201646
minute_NN  -  0.1765232350500377
every_DT  - 

weekend_NN  -  0.0
stayed_VBN  -  0.0
time_NN  -  0.0
hotel_NNS  -  0.0
location_NN  -  0.0
looking_VBG  -  0.0
staff_NN  -  0.0
vacation_NN  -  0.0
went_VBD  -  0.0
great_JJ  -  0.0
food_NN  -  0.0
nice_JJ  -  0.0
park_NN  -  0.0
grant_NN  -  0.0
michigan_NNS  -  0.0
anyone_NN  -  0.1364047373656643
would_MD  -  0.0704308732637901
hotel_NN  -  0.04474814849341855
pier_NN  -  0.0
navy_VB  -  0.0
lovely_RB  -  0.0
weather_NN  -  0.0
numerous_JJ  -  0.0
back_RB  -  0.0
treated_VBD  -  0.0
decision_NN  -  0.0
made_VBD  -  0.0
honestly_RB  -  0.0
price_NNS  -  0.0
exorbitant_JJ  -  0.0
pay_VB  -  0.0
without_IN  -  0.0
luxurious_JJ  -  0.0
last_JJ  -  0.0
trip_NN  -  0.0
well_RB  -  0.0
next_JJ  -  0.0
bed_NNS  -  0.0
wanted_VBD  -  0.0
family_NN  -  0.0
chose_VBD  -  0.0
definitely_RB  -  0.0
time_NN  -  0.0
say_VB  -  0.0
chicago_JJ  -  0.0
city_NN  -  0.0
something_NN  -  0.0
clean_JJ  -  0.0
staff_NN  -  0.0
attraction_NNS  -  0.0
great_JJ  -  0.0
excellent_JJ  -  0.0
room_NNS  -  0.0


james_NNS  -  0.0
since_IN  -  0.0
found_VBN  -  0.0
desk_NN  -  0.0
front_NN  -  0.0
last_JJ  -  0.0
away_RB  -  0.0
ambiance_NN  -  0.0
trip_NN  -  0.0
extremely_RB  -  0.0
life_NN  -  0.0
ever_RB  -  0.0
definitely_RB  -  0.0
bed_NN  -  0.0
chance_NN  -  0.0
comfortable_JJ  -  0.0
stayed_VBN  -  0.0
time_NN  -  0.0
say_VB  -  0.0
stay_NN  -  0.0
chicago_JJ  -  0.0
something_NN  -  0.0
friendly_JJ  -  0.0
clean_JJ  -  0.0
location_NN  -  0.0
thing_NNS  -  0.0
better_JJR  -  0.0
amenity_NNS  -  0.0
even_RB  -  0.0
perfect_JJ  -  0.0
staff_NN  -  0.0
pleasure_NN  -  0.0
business_NN  -  0.0
area_NN  -  0.0
great_JJ  -  0.0
room_NN  -  0.0
get_VB  -  0.0
day_NN  -  0.0
room_NNS  -  0.0
everything_NN  -  0.0
hotel_NN  -  0.04474814849341855
like_IN  -  0.09288320072309046
shampoo_NN  -  0.40289626389396044
bathroom_NN  -  0.1047373517617891
staying_VBG  -  0.22268074696654871
could_MD  -  0.08884482593784591
also_RB  -  0.09875666716213397
minute_NN  -  0.1765232350500377
enjoyed_VBN  -  

best_JJS  -  0.0
night_NN  -  0.0
even_RB  -  0.0
staff_NN  -  0.0
visiting_VBG  -  0.0
got_VBD  -  0.0
area_NN  -  0.0
great_JJ  -  0.0
room_NN  -  0.0
good_JJ  -  0.0
day_NN  -  0.0
place_NN  -  0.0
amazing_VBG  -  0.0
service_NN  -  0.07803712989209645
hotel_NN  -  0.04474814849341855
friend_NNS  -  0.1597969599098303
stay_VB  -  0.08075540064881251
chicago_NN  -  0.13440561908174142
four_CD  -  0.0
reasonable_JJ  -  0.0
book_NN  -  0.0
requested_VBN  -  0.0
prompt_JJ  -  0.0
star_NN  -  0.0
monaco_NN  -  0.0
allowed_VBN  -  0.0
member_NN  -  0.0
tell_VB  -  0.0
customer_NN  -  0.0
visit_NN  -  0.0
since_IN  -  0.0
liked_VBD  -  0.0
price_NNS  -  0.0
pet_JJ  -  0.0
well_RB  -  0.0
extremely_RB  -  0.0
next_JJ  -  0.0
friendly_RB  -  0.0
definitely_RB  -  0.0
time_NN  -  0.0
clean_JJ  -  0.0
even_RB  -  0.0
staff_NN  -  0.0
great_JJ  -  0.0
food_NN  -  0.0
t_RB  -  0.0
pet_NNS  -  0.0
nice_JJ  -  0.0
room_NNS  -  0.0
service_NN  -  0.07803712989209645
hotel_NN  -  0.04474814849341855

provided_VBD  -  0.0
efficient_JJ  -  0.0
happy_JJ  -  0.0
pleased_JJ  -  0.0
bed_NN  -  0.0
warm_JJ  -  0.0
see_VB  -  0.0
stay_NN  -  0.0
got_VBD  -  0.0
room_NN  -  0.0
overall_JJ  -  0.0
hotel_NN  -  0.04474814849341855
shampoo_NN  -  0.40289626389396044
bathroom_NN  -  0.1047373517617891
also_RB  -  0.09875666716213397
brought_VBD  -  0.0
request_NN  -  0.0
plenty_NN  -  0.0
enjoyable_JJ  -  0.0
reasonable_JJ  -  0.0
room_JJ  -  0.0
wine_NN  -  0.0
monaco_NN  -  0.0
cool_JJ  -  0.0
thing_NN  -  0.0
member_NN  -  0.0
another_DT  -  0.0
hour_NN  -  0.0
kid_NNS  -  0.0
spa_NN  -  0.0
since_IN  -  0.0
upon_IN  -  0.0
recently_RB  -  0.0
helpful_JJ  -  0.0
definitely_RB  -  0.0
especially_RB  -  0.0
really_RB  -  0.0
fantastic_JJ  -  0.0
stay_NN  -  0.0
chicago_JJ  -  0.0
friendly_JJ  -  0.0
downtown_NN  -  0.0
staff_NN  -  0.0
price_NN  -  0.0
got_VBD  -  0.0
area_NN  -  0.0
room_NN  -  0.0
anyone_NN  -  0.1364047373656643
recommend_VB  -  0.11039160645661797
would_MD  -  0.0704308732

...

like_IN  -  0.09288320072309046
could_MD  -  0.08884482593784591
chicago_NN  -  0.13440561908174142
computer_NN  -  0.0
apparently_RB  -  0.0
temperature_NN  -  0.0
lack_NN  -  0.0
total_JJ  -  0.0
horrible_JJ  -  0.0
tiny_JJ  -  0.0
despite_IN  -  0.0
photo_NN  -  0.0
motel_NN  -  0.0
privacy_NN  -  0.0
particular_JJ  -  0.0
nicer_NN  -  0.0
low_JJ  -  0.0
early_JJ  -  0.0
multiple_JJ  -  0.0
premium_JJ  -  0.0
fi_JJ  -  0.0
moved_VBN  -  0.0
5th_JJ  -  0.0
bar_NNS  -  0.0
cost_NN  -  0.0
look_VBZ  -  0.0
mine_NN  -  0.0
noise_NN  -  0.0
remote_NN  -  0.0
issue_NN  -  0.0
chair_NN  -  0.0
cheap_JJ  -  0.0
recommended_VBN  -  0.0
ride_NN  -  0.0
music_NN  -  0.0
corner_NN  -  0.0
lounge_NN  -  0.0
bottle_NN  -  0.0
street_NN  -  0.0
hour_NNS  -  0.0
rating_NN  -  0.0
huge_JJ  -  0.0
wi  -  0.0
car_NN  -  0.0
shower_NN  -  0.0
window_NN  -  0.0
cab_NN  -  0.0
floor_NN  -  0.0
line_NN  -  0.0
least_JJS  -  0.0
idea_NN  -  0.0
channel_NN  -  0.0
door_NN  -  0.0
think_VB  -  0.0
people_NNS

half_NN  -  0.0
sunday_JJ  -  0.0
september_NN  -  0.0
cost_NN  -  0.0
bug_NNS  -  0.0
worth_JJ  -  0.0
basically_RB  -  0.0
cleaning_NN  -  0.0
dry_JJ  -  0.0
honestly_RB  -  0.0
home_NN  -  0.0
come_VBN  -  0.0
re_VBP  -  0.0
bed_NN  -  0.0
night_NN  -  0.0
got_VBD  -  0.0
stayed_VBD  -  0.0
t_RB  -  0.0
hotel_NN  -  0.04474814849341855
friend_NNS  -  0.1597969599098303
stay_VB  -  0.08075540064881251
noisy_JJ  -  0.0
bother_VB  -  0.0
today_NN  -  0.0
intercontinental_NN  -  0.0
bed_JJ  -  0.0
hair_NN  -  0.0
next_IN  -  0.0
instead_RB  -  0.0
elevator_NN  -  0.0
in_JJ  -  0.0
check  -  0.0
requested_VBD  -  0.0
reservation_NN  -  0.0
person_NN  -  0.0
complimentary_JJ  -  0.0
must_MD  -  0.0
first_JJ  -  0.0
travel_VBP  -  0.0
often_RB  -  0.0
size_NN  -  0.0
king_NN  -  0.0
given_VBN  -  0.0
last_JJ  -  0.0
away_RB  -  0.0
chicago_VB  -  0.0
bed_NN  -  0.0
bad_JJ  -  0.0
business_NN  -  0.0
room_NN  -  0.0
t_RB  -  0.0
day_NN  -  0.0
service_NN  -  0.07803712989209645
deliver_VB  

apology_NN  -  0.0
confirmation_NN  -  0.0
changed_VBD  -  0.0
rudely_RB  -  0.0
supposed_VBN  -  0.0
sit_VB  -  0.0
directly_RB  -  0.0
intercontinental_NN  -  0.0
intercontinental_JJ  -  0.0
number_NN  -  0.0
instead_RB  -  0.0
told_VBD  -  0.0
figured_VBD  -  0.0
treated_VBN  -  0.0
concern_NN  -  0.0
telling_VBG  -  0.0
clear_JJ  -  0.0
married_VBN  -  0.0
cab_NN  -  0.0
booked_VBD  -  0.0
though_IN  -  0.0
another_DT  -  0.0
happy_JJ  -  0.0
respect_NN  -  0.0
special_JJ  -  0.0
ve_VBP  -  0.0
take_VB  -  0.0
everyone_NN  -  0.0
couple_NN  -  0.0
elegant_JJ  -  0.0
highly_RB  -  0.0
given_VBN  -  0.0
find_VB  -  0.0
never_RB  -  0.0
wedding_NN  -  0.0
night_NN  -  0.0
even_RB  -  0.0
got_VBD  -  0.0
went_VBD  -  0.0
room_NN  -  0.0
experience_NN  -  0.0
place_NN  -  0.0
anyone_NN  -  0.1364047373656643
recommend_VB  -  0.11039160645661797
would_MD  -  0.0704308732637901
hotel_NN  -  0.04474814849341855
stay_VB  -  0.08075540064881251
husband_NN  -  0.12838818282036576
reserved_VBN

best_JJS  -  0.0
guest_NNS  -  0.0
would_MD  -  0.0704308732637901
hotel_NN  -  0.04474814849341855
peeling_VBG  -  0.0
wallpaper_NN  -  0.0
peninsula_NN  -  0.0
little_RB  -  0.0
lockwood_NN  -  0.0
palmer_NN  -  0.0
ice_NN  -  0.0
par_NN  -  0.0
omni_NN  -  0.0
hyatt_NN  -  0.0
tired_JJ  -  0.0
michigan_JJ  -  0.0
barely_RB  -  0.0
better_RBR  -  0.0
lounge_NN  -  0.0
200_CD  -  0.0
plus_NN  -  0.0
paying_VBG  -  0.0
show_VB  -  0.0
ordered_VBD  -  0.0
small_JJ  -  0.0
excited_VBN  -  0.0
deluxe_JJ  -  0.0
thing_NN  -  0.0
quiet_JJ  -  0.0
house_NN  -  0.0
feel_VB  -  0.0
may_MD  -  0.0
way_NN  -  0.0
closet_NN  -  0.0
ave_VBP  -  0.0
suite_NN  -  0.0
executive_NN  -  0.0
well_RB  -  0.0
re_VBP  -  0.0
bed_NN  -  0.0
really_RB  -  0.0
something_NN  -  0.0
location_NN  -  0.0
better_JJR  -  0.0
view_NN  -  0.0
room_NN  -  0.0
restaurant_NNS  -  0.0
get_VB  -  0.0
nice_JJ  -  0.0
park_NN  -  0.0
bathroom_NN  -  0.1047373517617891
could_MD  -  0.08884482593784591
charged_VBD  -  0.0
dis

sitting_VBG  -  0.0
turn_VB  -  0.0
tax_NN  -  0.0
open_VB  -  0.0
tried_VBD  -  0.0
today_NN  -  0.0
understand_VB  -  0.0
red_JJ  -  0.0
tip_NN  -  0.0
palmer_NN  -  0.0
toiletry_NNS  -  0.0
machine_NN  -  0.0
unfortunately_RB  -  0.0
king_VBG  -  0.0
slightly_RB  -  0.0
reasonably_RB  -  0.0
duty_NN  -  0.0
left_VBN  -  0.0
condition_NN  -  0.0
saw_VBD  -  0.0
institute_NN  -  0.0
tasted_VBN  -  0.0
hyatt_NN  -  0.0
cheapest_JJS  -  0.0
ago_RB  -  0.0
millennium_NN  -  0.0
wall_NNS  -  0.0
cost_NN  -  0.0
expected_VBN  -  0.0
back_NN  -  0.0
chair_NN  -  0.0
built_VBN  -  0.0
lounge_NN  -  0.0
complaint_NN  -  0.0
bottle_NN  -  0.0
around_RB  -  0.0
hope_VBP  -  0.0
three_CD  -  0.0
extra_JJ  -  0.0
10  -  0.0
although_IN  -  0.0
along_IN  -  0.0
main_JJ  -  0.0
classic_JJ  -  0.0
changed_VBN  -  0.0
menu_NN  -  0.0
look_NN  -  0.0
ordered_VBD  -  0.0
hard_JJ  -  0.0
small_JJ  -  0.0
worth_JJ  -  0.0
lot_NN  -  0.0
expensive_JJ  -  0.0
space_NN  -  0.0
sized_JJ  -  0.0
evening_NN  -

In [113]:
import numpy as np
feature_array = np.array(vector.get_feature_names())
tfidf_sorting = np.argsort(cntvct_token.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]

In [100]:
print(tfidf_sorting)

[1546 7818 2030 ... 8675 8674    0]


In [114]:
def train_rfc(X,y):
    n_estimators = [100]
    min_samples_split = [2]
    min_samples_leaf = [1]
    bootstrap = [True]
    parameters = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf,
                  'min_samples_split': min_samples_split}
    clf = GridSearchCV(RandomForestClassifier(verbose=1,n_jobs=-1), cv=4, param_grid=parameters)
    clf.fit(X, y)
    return clf

In [136]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(cntvct_token, data["target"], test_size=0.3, random_state=2016)
rfc_clf = train_rfc(X_train,y_train)
print ("Accuracy of RF on CV sets :{}".format(rfc_clf.best_score_))
print(clf.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    3.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s fin

Accuracy of RF on CV sets :0.6982142857142857


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.6s finished


NameError: name 'clf' is not defined

In [137]:
print(rfc_clf.best_params_)

{'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [117]:
print("Accuracy of RF on test sets is : {}".format(rfc_clf.score(X_test,y_test)))

Accuracy of RF on test sets is : 0.7166666666666667


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [122]:
from sklearn.neighbors import KNeighborsClassifier
for k in range(1,31,1):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train,y_train)
    print(k,knn.score(X_test,y_test))
                               


1 0.5625
2 0.5125
3 0.5645833333333333
4 0.6104166666666667
5 0.6166666666666667
6 0.6125
7 0.6375
8 0.6416666666666667
9 0.65
10 0.6333333333333333
11 0.6479166666666667
12 0.6625
13 0.6479166666666667
14 0.6770833333333334
15 0.6833333333333333
16 0.6916666666666667
17 0.6895833333333333
18 0.6770833333333334
19 0.6979166666666666
20 0.7125
21 0.71875
22 0.7041666666666667
23 0.7145833333333333
24 0.70625
25 0.7083333333333334
26 0.71875
27 0.71875
28 0.7125
29 0.7083333333333334
30 0.7125


In [123]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train,y_train)
mlp.score(X_test,y_test)

0.7979166666666667

In [133]:
from sklearn.semi_supervised import LabelPropagation
label_prop_model = LabelPropagation()
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(len(y_train)) < 0.3
labels = np.copy(y_train)
labels[random_unlabeled_points] = -1
label_prop_model.fit(X_train.toarray(), labels)

LabelPropagation(gamma=20, kernel='rbf', max_iter=1000, n_jobs=None,
                 n_neighbors=7, tol=0.001)

In [134]:
label_prop_model.score(X_test.toarray(),y_test)

0.6375

In [135]:
from sklearn.semi_supervised import LabelSpreading
label_spread_model = LabelSpreading()
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(len(y_train)) < 0.3
labels = np.copy(y_train)
labels[random_unlabeled_points] = -1
label_spread_model.fit(X_train.toarray(), labels)
label_spread_model.score(X_test.toarray(),y_test)

0.65

In [163]:
with open("test.txt") as file:
        data = file.read()
        review_tokenized = []
        print(data)
        lmt = WordNetLemmatizer()
        tokenize_words = word_tokenize(data.lower(),language='english')
        print(tokenize_words)
        pos_word = pos_tag(tokenize_words)
        tokenize_words = ["_".join([lmt.lemmatize(i[0]),i[1]]) for i in pos_word if (i[0] not in stopwords.words("english") and len(i[0]) > 2)]
        token = ' '.join(tokenize_words)
        review_tokenized.append(token)
        res = vector.transform(review_tokenized)
        res = res.toarray() 
        pred = mlp.predict(res) 

Hotel rooms are not good. Cleaning is not good
['hotel', 'rooms', 'are', 'not', 'good', '.', 'cleaning', 'is', 'not', 'good']
hotel_NN room_NNS good_JJ cleaning_NN good_JJ <class 'str'>
[[0. 0. 0. ... 0. 0. 0.]] (1, 2000) <class 'numpy.ndarray'>
[3]
